In [1]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
from lightning.pytorch.trainer.states import TrainerFn

In [2]:
import os
from glob import glob

In [3]:
participants = glob('../data/scaleograms/**')

In [4]:
participants = [x.split('\\')[1] for x in participants]

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
train_participants, test_participants = train_test_split(participants, test_size=0.2, random_state=42)
val_participants, test_participants = train_test_split(test_participants, test_size=0.2, random_state=42)

In [7]:
import datasets
from datasets import load_dataset

In [8]:
load_dataset(
    './data/scaleograms', 
    trust_remote_code=True,
    train_participants=train_participants[:1], 
    val_participants=val_participants[:1], 
    test_participants=test_participants[:1],
    num_proc=8
)

FileNotFoundError: Couldn't find a dataset script at d:\university\thesis\experiments\data\scaleograms\scaleograms.py or any data file in the same directory.

In [9]:
# load_dataset(
#     './data/ecg', 
#     trust_remote_code=True,
#     train_participants=train_participants[:1], 
#     val_participants=val_participants[:1], 
#     test_participants=test_participants[:1],
#     column_names=['signal', 'label'],
#     num_proc=8
# )

In [8]:
ds = load_dataset(
    "csv", 
    data_files="../data/ecg_preprocessed/30100.csv", 
    usecols=["ECG_Clean", "category"], 
    features=datasets.Features({'ECG_Clean': datasets.Value('float64'), 'category': datasets.Value('string')}),
    num_proc=8
)
ds

DatasetDict({
    train: Dataset({
        features: ['ECG_Clean', 'category'],
        num_rows: 8692000
    })
})

In [9]:
import neurokit2 as nk

In [10]:
dataset = ds['train']

In [ ]:
def rename(signal, label):
    return {
        'signal': signal,
        'label': label
    }

dataset = dataset.map(
    rename, 
    batched=True,
    features=datasets.Features({
        'signal': datasets.Value('float64'), 
        'label': datasets.Value('string')
    }),
    input_columns=['ECG_Clean', 'category'], 
    remove_columns=['ECG_Clean', 'category']
)

In [13]:
import numpy as np

In [14]:
dataset

Dataset({
    features: ['signal', 'label'],
    num_rows: 8692000
})

In [44]:
import pandas as pd 
a = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})
a.to_dict('list')

{'a': [1, 2, 3], 'b': [4, 5, 6]}

In [46]:
def neurokit(sampling_rate: int = 1000):
    def inner(signal):
        df, _ = nk.ecg_process(signal, sampling_rate=sampling_rate, method='neurokit')
        return df.to_dict('list')
    return inner

dataset.map(
    neurokit(),
    batched=True,
    batch_size=int(len(dataset) * .25),
    input_columns=['signal'], 
)

Map: 100%|██████████| 8692000/8692000 [01:08<00:00, 127156.09 examples/s]


Dataset({
    features: ['signal', 'label', 'ECG_Raw', 'ECG_Clean', 'ECG_Rate', 'ECG_Quality', 'ECG_R_Peaks', 'ECG_P_Peaks', 'ECG_P_Onsets', 'ECG_P_Offsets', 'ECG_Q_Peaks', 'ECG_R_Onsets', 'ECG_R_Offsets', 'ECG_S_Peaks', 'ECG_T_Peaks', 'ECG_T_Onsets', 'ECG_T_Offsets', 'ECG_Phase_Atrial', 'ECG_Phase_Completion_Atrial', 'ECG_Phase_Ventricular', 'ECG_Phase_Completion_Ventricular'],
    num_rows: 8692000
})

In [ ]:
for value in dataset:
    print(value)
    break

In [47]:
import inspect

In [51]:
inspect.getfullargspec(neurokit(1000))

FullArgSpec(args=['signal'], varargs=None, varkw=None, defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})

In [58]:
def _get_args_dict(fn):
    return fn.__code__.co_varnames[:fn.__code__.co_argcount]

tttt

In [15]:
from glob import glob 
glob('../data/scaleograms/**/')[0].split('\\')[0]

'../data/scaleograms'

In [4]:
import datasets
from datasets import load_dataset

def _get_args(fn):
    return fn.__code__.co_varnames[:fn.__code__.co_argcount]

dataset = load_dataset(
    '../data/scaleograms/**', 
    trust_remote_code=True,
    num_proc=8
)

def resize(width: int, height: int):
    def inner(image):
        return {
            'pixel_values': [x.convert("RGB").resize((width, height)) for x in image]
        }
    return inner

dataset.map(
    resize(127, 127),
    batched=True,
    batch_size=int(len(dataset) * .25),
    input_columns=_get_args(resize(127, 127))
)


Map:   0%|          | 0/91408 [00:00<?, ? examples/s]

Map:   0%|          | 0/91408 [03:32<?, ? examples/s]


KeyboardInterrupt: 